# Recurrent Neural Networks
The following Recurrent Neural Networks (RNN) will be used for musical genre classification. This is because the task of classify all the new music that is released nowadays it is impossible to be done by a human being.

## Initialization

### Libraries

In [ ]:
# Import general purpose python libraries
import os
from pathlib import Path
import numpy as np
import tensorflow as tf
import keras
from keras.layers import LSTM, Bidirectional, Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import image_dataset_from_directory
! pip install keras-self-attention
from keras_self_attention import SeqSelfAttention

import matplotlib.pyplot as plt

# Import function to plot the results
!pip install seaborn
import plots

### Data Configuration Parameters
Configuration variables related to the data

In [ ]:
# Randomize the initial network weights
random_seed = True

# Paths to where training, testing, and validation images are
database_dir = 'dataset_multilabel'
train_dir = f'{database_dir}/training/spectrogram'
val_dir = f'{database_dir}/val/spectrogram'
test_dir = f'{database_dir}/test/spectrogram'

# Directory where to store weights of the model and results
root_dir = "results"
# Create root directory for results if it does not exist
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

# Input dimension (number of subjects in our problem)
num_classes = 6

# Name of each gesture of the database
# CLASSES = [x for x in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, x))]
CLASSES = ['Alternative', 'Pop', 'Rock', 'Dance', 'Classical', 'Techno']
print(f'The classess to classify are: {CLASSES}')

# Parameters that characterise the spectrogram
img_height = 369
img_width = 496
img_channels = 1
color_mode = 'grayscale'

### Configuration Training Parameters

In [ ]:
# Parameters that configures the training process
batch_size = 20  # Batch size
epochs = 50  # Number of epochs
initial_lr = 1e-5   # Learning rate
seed = 42  # Random number
num_layers = 4
neu1 = 64
neu2 = 128
neu3 = 512
modelCNN = 'VGG'  # LSTM or BLSTM
version = ''
if num_layers == 2:
    version = f'BS{batch_size}_E{epochs}_LR{initial_lr}_Layers{num_layers}_NueronsL1{neu1}_NueronsL2{neu2}'
elif num_layers == 1:
    version = f'BS{batch_size}_E{epochs}_LR{initial_lr}_Layers{num_layers}_NueronsL1{neu1}'
elif num_layers == 3:
    version = f'BS{batch_size}_E{epochs}_LR{initial_lr}_Layers{num_layers}_NueronsL1{neu1}_NueronsL2{neu2}_NueronsL3{neu3}'
else:
    version = f'BS{batch_size}_E{epochs}_LR{initial_lr}_Layers{num_layers}_ALL'
experiment_dir = f'{root_dir}/Multilabel_{modelCNN}'

# Create experiment directory if it does not exist
if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

# Set random seed
np.random.seed(seed)
tf.random.set_seed(seed)

### Loading of training, validation and test datasets of images
1.   Training dataset
2.   Validation dataset
3.   Test dataset

In [ ]:
# Training labels
train_df = pd.read_csv(f'{database_dir}/training_multilabel.csv', delimiter='\t')
train_df['Genres']= train_df['Genres'].apply(lambda x:x.split(", "))
# train_df['Genres'] = mlb.fit_transform(train_df['Genres'])
train_df['TRACK_ID'] = train_df['TRACK_ID'].astype(str)

# Validation labels
val_df = pd.read_csv(f'{database_dir}/validation_multilabel.csv', delimiter='\t')
val_df['Genres']= val_df['Genres'].apply(lambda x:x.split(", "))
val_df['TRACK_ID'] = val_df['TRACK_ID'].astype(str)

# Test labels
test_df = pd.read_csv(f'{database_dir}/test_multilabel.csv', delimiter='\t')
test_df['Genres'] = test_df['Genres'].apply(lambda x:x.split(", "))
test_df['TRACK_ID'] = test_df['TRACK_ID'].astype(str)

In [ ]:
train_generator=ImageDataGenerator().flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col='TRACK_ID',
    y_col='Genres',
    batch_size=batch_size,
    seed=seed,
    shuffle=True,
    class_mode='categorical',
    classes=CLASSES,
    target_size=(img_height, img_width))

val_generator=ImageDataGenerator().flow_from_dataframe(
    dataframe=val_df,
    directory=val_dir,
    x_col='TRACK_ID',
    y_col='Genres',
    batch_size=batch_size,
    seed=seed,
    shuffle=True,
    class_mode='categorical',
    classes=CLASSES,
    target_size=(img_height, img_width))

test_generator=ImageDataGenerator().flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='TRACK_ID',
    y_col='Genres',
    batch_size=batch_size,
    seed=seed,
    shuffle=False,
    class_mode='categorical',
    classes=CLASSES,
    target_size=(img_height, img_width))

### Example of a sprectrogram from the training dataset

In [ ]:
images, labels = train_generator.next()

# Muestra la primera imagen del lote
plt.imshow(images[0].astype('uint8'))
plt.axis('off')  # Desactiva los ejes
plt.show()
print(images[0].shape)

## Training process
#### Available Models: LSTM & BiLSTM

### Long Short-Term Memory (LSTM)

In [ ]:
def lstm(height, width, out_dim, neurons):
    # Sequential Model
    model_lstm = Sequential()
    model_lstm.add(LSTM(neurons, input_shape=(height, width), activation='tanh', return_sequences=True))
    model_lstm.add(SeqSelfAttention(attention_activation='sigmoid'))
    #model_lstm.add(Flatten())
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(32, activation='tanh'))
    model_lstm.add(SeqSelfAttention(attention_activation='sigmoid'))
    model_lstm.add(Dropout(0.2))

    model_lstm.add(Dense(32, activation='relu'))
    model_lstm.add(Dropout(0.2))

    model_lstm.add(Dense(out_dim, activation='softmax'))

    return model_lstm

### Bidirectional Long Short-Term Memory (BiLSTM)

In [ ]:
def bi_lstm(height, width, out_dim, neurons):
    # Sequential Model
    model_bilstm = Sequential()
    model_bilstm.add(Bidirectional(LSTM(neurons, input_shape=(height, width), activation='tanh', return_sequences=True)))
    model_bilstm.add(SeqSelfAttention(attention_activation='sigmoid'))
    #model_bilstm.add(Flatten())
    model_bilstm.add(Dropout(0.2))

    model_bilstm.add(Bidirectional(LSTM(neurons, activation='tanh')))
    model_bilstm.add(SeqSelfAttention(attention_activation='sigmoid'))
    model_bilstm.add(Dropout(0.2))

    model_bilstm.add(Dense(64, activation='relu'))
    model_bilstm.add(Dropout(0.2))

    model_bilstm.add(Dense(out_dim, activation='softmax'))

    # Explicitly build the model
    model_bilstm.build(input_shape=(None,height, width))
    
    return model_bilstm

## Model execution

In [ ]:
#Model
model = None
if modelRNN == 'LSTM':
    model = lstm(img_height, img_width, num_classes, num_neurons)
elif modelRNN == 'BiLSTM':
    model = bi_lstm(img_height, img_width, num_classes, num_neurons)
else:
    print('Wrong model selection or Model no available\n')

# Print the architecture of the model
model.summary()

## Set model training process
#### Configuration of several training decisions:
1. Optimizer using `Adam`
2. Model training configuration using `compile` with `binary_crossentropy` due to the classification labeling

In [ ]:
# 1. Configure optimizer
adam = Adam(learning_rate=initial_lr)

# 2. Configure training process
model.compile(loss = ['binary_crossentropy'],optimizer=adam, metrics=['accuracy'])

## Train the model
1. Load parameters from previous trainings if they exist.
2. Fit the model
3. Save the weights

In [ ]:
# Load pretrained model
weights_path = f"weights_{version}.h5" # Name of the file to store the weights
weights_file = Path(weights_path)
weights_load_path = f'{experiment_dir}/{weights_path}'
#if weights_load_path:
#    try:
#        model.load_weights(weights_load_path)
#        print("Loaded model from {}".format(weights_load_path))
#    except:
#        print("Impossible to find weight path. Returning untrained model")

# Fit the model
history = model.fit(train_generator, validation_data=val_generator, epochs=epochs, batch_size=batch_size)

# Save weights
# weights_save_path = os.path.join(experiment_dir, weights_path)
# model.save_weights(weights_save_path)

## Training Results
Accuracy and Loss obtained along the training process

In [ ]:
plots.accloss(history, modelCNN, experiment_dir, version)

## Testing
### Model Testing
1. Compute the loss function and accuracy for the test data
2. Confusion Matrix obtained from testing results

In [ ]:
# y_test
mlb = MultiLabelBinarizer()
y_test = test_generator.labels
y_test = mlb.fit_transform(y_test)

# Evaluate model
scores = model.evaluate(test_generator, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
print("Loss: %.2f" % scores[0])

# Obtain results to present the confusion matrix
prob_class = model.predict(test_generator, batch_size=batch_size)
# Convert the probabilities into binary classes
threshold = 0.5 
y_pred = tf.cast(tf.math.greater_equal(prob_class, threshold), tf.int32)

# Classification Report
report = classification_report(y_test, y_pred, output_dict=False, target_names=CLASSES)
print(report)

# Visualize confusion matrix                                           
plots.cm_mutilabel(y_test, y_pred, modelCNN, CLASSES, experiment_dir, version)